In [1]:
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import os
import pickle
import time
from tqdm.notebook import tqdm

import torch
torch.set_default_tensor_type(torch.DoubleTensor)

from spatial_scene_grammars.nodes import *
from spatial_scene_grammars.rules import *
from spatial_scene_grammars.scene_grammar import *
from spatial_scene_grammars.visualization import *
from spatial_scene_grammars_examples.table.grammar import *
from spatial_scene_grammars.parsing import *
from spatial_scene_grammars.sampling import *
from spatial_scene_grammars.parameter_estimation import *
from spatial_scene_grammars.dataset import *

import meshcat
import meshcat.geometry as meshcat_geom

In [2]:
if 'vis' not in globals():
    vis = meshcat.Visualizer()
vis.delete()
base_url = "http://127.0.0.1"
meshcat_url = base_url + ":" + vis.url().split(":")[-1]
print("Meshcat url: ", meshcat_url)
'''
from IPython.display import HTML
HTML("""
    <div style="height: 400px; width: 100%; overflow-x: auto; overflow-y: hidden; resize: both">
    <iframe src="{url}" style="width: 100%; height: 100%; border: none"></iframe>
</div>
""".format(url=meshcat_url))
'''

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7005/static/
Meshcat url:  http://127.0.0.1:7005/static/


'\nfrom IPython.display import HTML\nHTML("""\n    <div style="height: 400px; width: 100%; overflow-x: auto; overflow-y: hidden; resize: both">\n    <iframe src="{url}" style="width: 100%; height: 100%; border: none"></iframe>\n</div>\n""".format(url=meshcat_url))\n'

In [3]:
# Set up grammar
grammar = SpatialSceneGrammar(
    root_node_type = Table,
    root_node_tf = drake_tf_to_torch_tf(RigidTransform(p=[0.0, 0., 0.8]))
)

In [4]:
# Constraints
constraints = [
    ObjectsOnTableConstraint(),
    ObjectSpacingConstraint(),
    TallStackConstraint()
]

# Sample a tree that satisfies the discrete constraints
max_attempts = 1000
tree, success = rejection_sample_under_constraints(
    grammar, [c for c in constraints if isinstance(c, StructureConstraint)],
    max_num_attempts=max_attempts, detach=True
)
if not success:
    logging.error("Didn't sample a good tree in %d attempts." % max_attempts)
hmc_tree = deepcopy(tree)
samples = do_fixed_structure_hmc_with_constraint_penalties(
    grammar, hmc_tree, num_samples=50, subsample_step=5,
    with_nonpenetration=False, zmq_url=vis.window.zmq_url,
    constraints=constraints,
    kernel_type="NUTS", max_tree_depth=6, target_accept_prob=0.8, step_size=1E-2, adapt_step_size=True,
    #kernel_type="HMC", num_steps=1, step_size=1E-4, adapt_step_size=False # Langevin-ish
)

0 tensor(inf)
1 tensor(3.)
9 tensor(1.)
51 tensor(1.)


[2022-01-24 16:55:59.656] [console] [warning] FindResource ignoring DRAKE_RESOURCE_ROOT='/home/gizatt/drake' because it does not contain a 'drake' subdirectory.


Initial trace log prob:  tensor(-4080.4869)
Connecting to meshcat-server at zmq_url=tcp://127.0.0.1:6005...
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7005/static/
Connected to meshcat-server.


Sample: 100%|██████████| 75/75 [02:04,  1.66s/it, step size=9.69e-02, acc. prob=0.286]


In [37]:
k=0
draw_scene_tree_contents_meshcat(samples[-k], zmq_url=vis.window.zmq_url,  prefix="hmc_samples/%d/contents" % k)
draw_scene_tree_structure_meshcat(samples[-k], zmq_url=vis.window.zmq_url,  prefix="hmc_samples/%d/structure" % k)

In [38]:
# Do final projeciton pass
final_tree = project_tree_to_feasibility(samples[-k], do_forward_sim=True, timestep=0.001, T=1.)
draw_scene_tree_contents_meshcat(samples[-k], zmq_url=vis.window.zmq_url,  prefix="final_sample/contents")
draw_scene_tree_structure_meshcat(samples[-k], zmq_url=vis.window.zmq_url,  prefix="final_sample/structure")

In [39]:
# Save that scene out for use in another notebook
with open("example_feasible_sampled_scene.pickle", "wb") as f:
    pickle.dump(final_tree, f)

# Random single scene parsing test

In [40]:
print("Ground truth tree score: ", tree.score())

Ground truth tree score:  tensor([-82.7861])


In [41]:
# Try to parse "tree" sampled above
# Parse it with IP.
vis["ip_parses"].delete()
supertree = grammar.make_super_tree(max_recursion_depth=10)
print("Supertree size ", len(supertree.nodes))

observed_nodes = tree.get_observed_nodes()
proposed_poses_by_type = generate_candidate_node_pose_sets(
    grammar, observed_nodes, max_recursion_depth=10, verbose=True
)
parse_trees = infer_mle_tree_with_mip_from_proposals(
    grammar, observed_nodes, {}, verbose=2, N_solutions=4
)
parse_tree = parse_trees[0]
for k, parse_tree in enumerate(parse_trees[:1]):
    score = parse_tree.score()
    print("SOL #%d: SCORE %f" % (k, score))
    draw_scene_tree_structure_meshcat(parse_tree, zmq_url=vis.window.zmq_url, prefix="ip_parses/%d_%f" % (k, score), linewidth=20, with_triad=False, node_sphere_size=0.025, alpha=0.5)

Supertree size  88
12 top-down candidates.
24 bottom-up candidates.
Post final pruning:  36
Optimization success?:  True
Logfile: 

Gurobi 9.0.2 (linux64) logging started Mon Jan 17 20:47:17 2022

Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (linux64)
Optimize a model with 1206 rows, 843 columns and 3838 nonzeros
Model fingerprint: 0xb721da2b
Variable types: 0 continuous, 843 integer (843 binary)
Coefficient statistics:
  Matrix range     [2e-02, 1e+00]
  Objective range  [7e-01, 5e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-02, 1e+00]
Presolve removed 1185 rows and 803 columns
Presolve time: 0.01s
Presolved: 21 rows, 40 columns, 77 nonzeros
Variable types: 0 continuous, 40 integer (40 binary)
Found heuristic solution: objective 82.7861243

Root relaxation: objective 8.278612e+01, 4 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0   

Node <spatial_scene_grammars_examples.table.grammar.Table object at 0x7f16248f2eb8> getting tf tensor([[1.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 1.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 1.0000, 0.8000],
        [0.0000, 0.0000, 0.0000, 1.0000]])
Node <spatial_scene_grammars_examples.table.grammar.PlaceSettings object at 0x7f16242460f0> getting tf tensor([[1.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 1.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 1.0000, 0.8250],
        [0.0000, 0.0000, 0.0000, 1.0000]])
Added Table_1794--(0)>PlaceSettings_1769
Node <spatial_scene_grammars_examples.table.grammar.SharedStuff object at 0x7f1624246e10> getting tf tensor([[1.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 1.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 1.0000, 0.8250],
        [0.0000, 0.0000, 0.0000, 1.0000]])
Added Table_1794--(1)>SharedStuff_1769
Node <spatial_scene_grammars_examples.table.grammar.PlaceSetting object at 0x7f16242464e0> getting tf tensor([[ 1.